# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://github.com/MachineLearningHW/ML_HW4_Dataset)

# Download dataset
- Data is [here](https://github.com/MachineLearningHW/ML_HW4_Dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
If the links below become inaccessible, please connect TAs.
"""
!unzip /content/drive/MyDrive/ML/HW4/ml2022spring-hw4.zip
#!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partaa
#!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partab
#!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partac
#!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partad


  inflating: Dataset/uttr-eca54d795c5a40a8b6d4c42e79f800cb.pt  
  inflating: Dataset/uttr-eca67d46a97a4d15bbfce9b0fdd25583.pt  
  inflating: Dataset/uttr-eca94cc2cc6346088cb250c1fe84513e.pt  
  inflating: Dataset/uttr-eca98e1f5b104cb99d740d628c3e45ae.pt  
  inflating: Dataset/uttr-eca99dbfac4b43e89bc3f0422e2ee470.pt  
  inflating: Dataset/uttr-eca9f3579bef4882bbe5f0830f7207d9.pt  
  inflating: Dataset/uttr-ecab285d2cb7468e9e33e12d7ded7e7a.pt  
  inflating: Dataset/uttr-ecabe30e9c39440187cae18e3d725de4.pt  
  inflating: Dataset/uttr-ecabefadc9c24a05b92b7be70faeb502.pt  
  inflating: Dataset/uttr-ecac19db51ff416ea72007e166e69e4b.pt  
  inflating: Dataset/uttr-ecacad40447a4ab88b8641688f37fdd2.pt  
  inflating: Dataset/uttr-ecaefd35fa534fa6b60216675e35d113.pt  
  inflating: Dataset/uttr-ecaf4ef0e4b0410db4379ff439bad87b.pt  
  inflating: Dataset/uttr-ecb1da5219bc4015ac38d93cadac81e8.pt  
  inflating: Dataset/uttr-ecb30cece7cf4153857ada6e2136b7ec.pt  
  inflating: Dataset/uttr-ecb3860fd5314

In [ ]:
#!cat Dataset.tar.gz.parta* > Dataset.tar.gz
#!tar zxvf Dataset.tar.gz

## Fix Random Seed

In [ ]:
import numpy as np
import torch
import random
import torchaudio
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len
	
		# Load the mapping from speaker neme to their corresponding id. 
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]
	
		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]
	
		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
	def __len__(self):
			return len(self.data)
 
	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker
 
	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [ ]:
# https://github.com/lucidrains/conformer
!pip install conformer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class SelfAttentionPooling(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SelfAttentionPooling, self).__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.attention_linear = nn.Linear(input_dim, self.hidden_dim)
        self.attention_activation = nn.Sigmoid()
        self.attention_vector = nn.Parameter(torch.rand(self.hidden_dim, 1))
        self.attention_soft = nn.Tanh()

    def forward(self, x):
        """
        :param x:   [batch, length, feat_dim] vector
        :return:   [batch, feat_dim] vector
        """
        x_shape = x.shape
        x = x.squeeze()
        if x_shape[0] == 1:
            x = x.unsqueeze(0)
        assert len(x.shape) == 3, print(x.shape)
        if x.shape[-2] == self.input_dim:
            x = x.transpose(-1, -2)
        # print(x.shape)
        if len(x.shape) == 4:
            x = x.reshape(x_shape[0], -1, x_shape[2])# , print(x.shape)
        if x.shape[-2] == self.input_dim:
            x = x.transpose(-1, -2)
        fx = self.attention_activation(self.attention_linear(x))
        vf = fx.matmul(self.attention_vector)
        alpha = self.attention_soft(vf)

        alpha_ht = x.mul(alpha)
        mean = torch.sum(alpha_ht, dim=-2)

        return mean

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdMSoftmaxLoss(nn.Module):

    def __init__(self, in_features, out_features, s=30.0, m=0.4):
        '''
        AM Softmax Loss
        '''
        super(AdMSoftmaxLoss, self).__init__()
        self.s = s
        self.m = m
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)

    def forward(self, x, labels):
        '''
        input shape (N, in_features)
        '''
        assert len(x) == len(labels)
        assert torch.min(labels) >= 0
        assert torch.max(labels) < self.out_features
        
        for W in self.fc.parameters():
            W = F.normalize(W, dim=1)

        x = F.normalize(x, dim=1)
        #print(x.shape)
        wf = self.fc(x)
        numerator = self.s * (torch.diagonal(wf.transpose(0, 1)[labels]) - self.m)
        excl = torch.cat([torch.cat((wf[i, :y], wf[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
        L = numerator - torch.log(denominator)
        return -torch.mean(L), wf

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
	def __init__(self, config, d_model=256, n_spks=600, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Change Transformer to Conformer.
		#   https://arxiv.org/abs/2005.08100
		'''self.encoder_layer = nn.TransformerEncoderLayer(
			d_model=d_model, dim_feedforward=256, nhead=2
		)'''
		# self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
		self.conformer = ConformerBlock(**config)
		#self.conformer2 = ConformerBlock(
		#	  dim = d_model, # input feature
		#	  dim_head = 64,
		#	  heads=2,
		#	  ff_mult=2, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
		#	  conv_expansion_factor=15, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
		#	  conv_kernel_size=31,
		#	  attn_dropout=0.2,
		#	  ff_dropout=0.2,
		#	  conv_dropout=0.2,
		#)
		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, 128),
		)
		self.attention_pooling = SelfAttentionPooling(d_model,256)
		self.adms_loss = AdMSoftmaxLoss(128,n_spks, s=30.0, m=0.2)

	def forward(self, mels, labels = None):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# out: (batch size, length, d_model)
		out = self.prenet(mels)
		# out: (length, batch size, d_model)
		out = out.permute(1, 0, 2)
		# The encoder layer expect features in the shape of (batch size,length,d_model).
		out = self.conformer(out)
		#out = self.conformer2(out)
		# out: (batch size, length, d_model)
		out = out.transpose(0, 1)
		# mean pooling
		#stats = out.mean(dim=1)

		stats = self.attention_pooling(out)
		#print(stats.shape)
		# out: (batch, n_spks)
		out = self.pred_layer(stats)
		loss = self.adms_loss(out,labels)
		return loss

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [ ]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [ ]:
import torch


def model_fn(batch, model,device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	loss, preds = model(mels, labels)
	#print(outs.shape)
	#loss = criterion(outs, labels)
	preds = preds.argmax(1)
	# Get the speaker id with highest probability.
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [ ]:
from tqdm import tqdm
import torch


def valid(dataloader, model,device): 
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	#pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch,model,device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		'''pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()'''
	print(f'Val: loss={running_loss / (i+1):.2f}, acc={running_accuracy / (i+1):.2f}')
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"batch_size": 64,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 150000,
		'model_config':{
			  "config1":{
				"dim":256, # input feature
			  	"dim_head":256,
			  	"heads":2,
			  	"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
			  	"conv_expansion_factor":25, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
			  	"conv_kernel_size":43,
			  	"attn_dropout":0.2,
			  	"ff_dropout":0.2,
			  	"conv_dropout":0.2,
			  },
			  "config2":{
				"dim":256, # input feature
			  	"dim_head":300,
			  	"heads":1,
			  	"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
			  	"conv_expansion_factor":20, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
			  	"conv_kernel_size":41,
			  	"attn_dropout":0.1,
			  	"ff_dropout":0.1,
			  	"conv_dropout":0.1,
			  },
			  "config3":{
				"dim":256, # input feature
			  	"dim_head":350,
			  	"heads":2,
			  	"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
			  	"conv_expansion_factor":22, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
			  	"conv_kernel_size":39,
			  	"attn_dropout":0.1,
			  	"ff_dropout":0.1,
			  	"conv_dropout":0.1,
			  }
		},
		"model_path":{
			  "config1":"/content/drive/MyDrive/ML/HW4/model1.ckpt",
			  "config2":"/content/drive/MyDrive/ML/HW4/model2.ckpt",
			  "config3":"/content/drive/MyDrive/ML/HW4/model3.ckpt",
		}
	}

	return config


def main(
	data_dir,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
	model_config,
	model_path,
):
	"""Main function."""
	for i in model_config:
		print(i)
		set_seed(87)
		device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
		print(f"[Info]: Use {device} now!")

		train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
		train_iterator = iter(train_loader)
		print(f"[Info]: Finish loading data!",flush = True)

		model = Classifier(model_config[i],n_spks=speaker_num).to(device)
		#criterion = nn.CrossEntropyLoss()
		optimizer = AdamW(model.parameters(), lr=1e-3)
		scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
		print(f"[Info]: Finish creating model!",flush = True)

		best_accuracy = -1.0
		best_state_dict = None

		#pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		for step in range(total_steps):
			# Get data
			try:
				batch = next(train_iterator)
			except StopIteration:
				train_iterator = iter(train_loader)
				batch = next(train_iterator)

			loss, accuracy = model_fn(batch, model,device)
			batch_loss = loss.item()
			batch_accuracy = accuracy.item()

			# Updata model
			loss.backward()
			optimizer.step()
			scheduler.step()
			optimizer.zero_grad()

			# Log
			'''pbar.update()
			pbar.set_postfix(
				loss=f"{batch_loss:.2f}",
				accuracy=f"{batch_accuracy:.2f}",
				step=step + 1,
			)'''

			# Do validation
			if (step + 1) % valid_steps == 0:
				print(f'Training: loss={batch_loss}, acc={batch_accuracy}')
				#pbar.close()

				valid_accuracy = valid(valid_loader,model,device)

				# keep the best model
				if valid_accuracy > best_accuracy:
					best_accuracy = valid_accuracy
					best_state_dict = model.state_dict()

				#pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

			# Save the best model so far.
			if (step + 1) % save_steps == 0 and best_state_dict is not None:
				torch.save(best_state_dict, model_path[i])
				print(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

		#pbar.close()


if __name__ == "__main__":
	main(**parse_args())

config1
[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!
Training: loss=9.023635864257812, acc=0.34375
Val: loss=9.54, acc=0.23
Training: loss=8.142646789550781, acc=0.40625
Val: loss=8.47, acc=0.37
Training: loss=7.1094889640808105, acc=0.578125
Val: loss=7.67, acc=0.45
Training: loss=7.8767595291137695, acc=0.453125
Val: loss=7.01, acc=0.51
Training: loss=6.79833459854126, acc=0.546875
Val: loss=6.38, acc=0.56
Step 10000, best model saved. (accuracy=0.5629)
Training: loss=5.7042951583862305, acc=0.65625
Val: loss=6.07, acc=0.59
Training: loss=4.998985290527344, acc=0.625
Val: loss=5.64, acc=0.63
Training: loss=5.831299781799316, acc=0.640625
Val: loss=5.45, acc=0.65
Training: loss=4.998030662536621, acc=0.640625
Val: loss=5.36, acc=0.66
Training: loss=3.941148042678833, acc=0.734375
Val: loss=5.08, acc=0.67
Step 20000, best model saved. (accuracy=0.6701)
Training: loss=3.610320806503296, acc=0.78125
Val: loss=4.90, acc=0.68
Training: loss=3.8363740444

*斜體文字*# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": {
			  "model1":"/content/drive/MyDrive/ML/HW4/model1.ckpt",
			  "model2":"/content/drive/MyDrive/ML/HW4/model2.ckpt",
			  "model3":"/content/drive/MyDrive/ML/HW4/model3.ckpt",
		},
		"output_path": "/content/drive/MyDrive/ML/HW4/output.csv",
		'model_config':{
			  "config1":{
				"dim":256, # input feature
			  	"dim_head":256,
			  	"heads":2,
			  	"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
			  	"conv_expansion_factor":25, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
			  	"conv_kernel_size":43,
			  	"attn_dropout":0.2,
			  	"ff_dropout":0.2,
			  	"conv_dropout":0.2,
			  },
			  "config2":{
				"dim":256, # input feature
			  	"dim_head":300,
			  	"heads":1,
			  	"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
			  	"conv_expansion_factor":20, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
			  	"conv_kernel_size":41,
			  	"attn_dropout":0.1,
			  	"ff_dropout":0.1,
			  	"conv_dropout":0.1,
			  },
			  "config3":{
				"dim":256, # input feature
			  	"dim_head":350,
			  	"heads":2,
			  	"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
			  	"conv_expansion_factor":22, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
			  	"conv_kernel_size":39,
			  	"attn_dropout":0.1,
			  	"ff_dropout":0.1,
			  	"conv_dropout":0.1,
			  }
		},
	}
	return config


def main(
	data_dir,
	model_path,
	output_path,
	model_config,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model1 = Classifier(model_config["config1"],n_spks=speaker_num).to(device)
	model1.load_state_dict(torch.load(model_path['model1']))
	model1.eval()
	model2 = Classifier(model_config["config2"],n_spks=speaker_num).to(device)
	model2.load_state_dict(torch.load(model_path['model2']))
	model2.eval()
	model3 = Classifier(model_config["config3"],n_spks=speaker_num).to(device)
	model3.load_state_dict(torch.load(model_path['model3']))
	model3.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			labels = torch.zeros(len(mels)).long().to(device)
			mels = mels.to(device)
			_, outs1 = model1(mels,labels)
			_, outs2 = model2(mels,labels)
			_, outs3 = model3(mels,labels)
			outs = (outs1+outs3+outs2)/3
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]

In [ ]:
# best acc = 0.78275
# 150000 steps
# ensemble with 3 config
# self-attention pooling
# margin softmax
"""
"config1":{
	  "dim":256, # input feature
	  "dim_head":256,
	  "heads":2,
	  "ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
      "conv_expansion_factor":25, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
	  "conv_kernel_size":43,
      "attn_dropout":0.2,
	  "ff_dropout":0.2,
	  "conv_dropout":0.2,
	},
"config2":{
	  "dim":256, # input feature
      "dim_head":300,
	  "heads":1,
	  "ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
	  "conv_expansion_factor":20, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
	  "conv_kernel_size":41,
	  "attn_dropout":0.1,
	  "ff_dropout":0.1,
	  "conv_dropout":0.1,
	},
"config3":{
	  "dim":256, # input feature
	  "dim_head":350,
	  "heads":2,
	  "ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))
	  "conv_expansion_factor":22, # inner_dim = dim*expansion_factor -> input_channel = inner_dim
	  "conv_kernel_size":39,
	  "attn_dropout":0.1,
	  "ff_dropout":0.1,
	  "conv_dropout":0.1,
	  }
"""

'\n\t\t\t  "config1":{\n\t\t\t\t"dim":256, # input feature\n\t\t\t  \t"dim_head":256,\n\t\t\t  \t"heads":2,\n\t\t\t  \t"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))\n\t\t\t  \t"conv_expansion_factor":25, # inner_dim = dim*expansion_factor -> input_channel = inner_dim\n\t\t\t  \t"conv_kernel_size":43,\n\t\t\t  \t"attn_dropout":0.2,\n\t\t\t  \t"ff_dropout":0.2,\n\t\t\t  \t"conv_dropout":0.2,\n\t\t\t  "config2":{\n\t\t\t\t"dim":256, # input feature\n\t\t\t  \t"dim_head":300,\n\t\t\t  \t"heads":1,\n\t\t\t  \t"ff_mult":4, # expand Linear module in feedforward by ff_mult (nn.Linear(dim,dim*ff_mult))\n\t\t\t  \t"conv_expansion_factor":20, # inner_dim = dim*expansion_factor -> input_channel = inner_dim\n\t\t\t  \t"conv_kernel_size":41,\n\t\t\t  \t"attn_dropout":0.1,\n\t\t\t  \t"ff_dropout":0.1,\n\t\t\t  \t"conv_dropout":0.1,\n\t\t\t  },\n\t\t\t  "config3":{\n\t\t\t\t"dim":256, # input feature\n\t\t\t  \t"dim_head":350,\n\t\t\t  \t"heads":2,\n\t\t\t 